Tutorial 3 Neural Networks
===

<br> Neural networks can be constructed using the ``torch.nn`` package.

<br> Now that you had a glimpse of ``autograd``, ``nn`` depends on autograd to define models and differentiate them. An ``nn.Module`` contains layers, and a method ``forward(input)``that returns the ``output``.

<br> In this example, we will look at a neural network that classifies digit images.

<br> A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network's parameters
- Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient


## 1. Define the network


In [1]:
# a network is defined like this:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        # defines the layer info in the __init__
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        # define the forward function in nn
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In the net class, you just have to define the ``forward`` function, and the ``backward`` function (where gradients are computed) is automatically defined for you using autograd. You can use any of the Tensor operations in the ``forward`` function.

<br> The learnable parameters of a model are returned by ``net.parameters()``

In [17]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


Let’s try a random 32x32 input. 

<br> *Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32. The random input is given by ``torch.randn(1, 1, 32, 32)``, where the first arg is the batch size, second number of input chanels, third and forth are size of the pic.*

In [1]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

# to auto get the gradients, first zero the gradient buffers of all parameters 
# and backprops with random gradients with ``net.zero_grad()``

net.zero_grad()
out.backward(torch.randn(1, 10))



NameError: name 'torch' is not defined

```
Notes:
``torch.nn`` only supports mini-batches. The entire ``torch.nn`` package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, ``nn.Conv2d`` will take in a 4D Tensor of nSamples x nChannels x Height x Width.

If you have a single sample, just use ``input.unsqueeze(0)`` to add a fake batch dimension.
```

## 2. Loss Function
A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package. A simple loss is: nn.MSELoss which computes the mean-squared error between the input and the target.